# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5fb4de0d90>
├── 'a' --> tensor([[ 0.5475, -0.0708,  0.0126],
│                   [-0.9566,  2.0148,  0.2394]])
└── 'x' --> <FastTreeValue 0x7f5fb4de0d60>
    └── 'c' --> tensor([[ 0.7988, -1.0082, -1.7218, -0.3509],
                        [-1.3154, -0.4027, -2.8071,  0.8890],
                        [ 1.2666, -0.2290,  1.1835,  2.4907]])

In [4]:
t.a

tensor([[ 0.5475, -0.0708,  0.0126],
        [-0.9566,  2.0148,  0.2394]])

In [5]:
%timeit t.a

65 ns ± 0.042 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5fb4de0d90>
├── 'a' --> tensor([[-0.9177,  0.3324, -0.2741],
│                   [-0.5876,  0.7655, -0.8117]])
└── 'x' --> <FastTreeValue 0x7f5fb4de0d60>
    └── 'c' --> tensor([[ 0.7988, -1.0082, -1.7218, -0.3509],
                        [-1.3154, -0.4027, -2.8071,  0.8890],
                        [ 1.2666, -0.2290,  1.1835,  2.4907]])

In [7]:
%timeit t.a = new_value

68.3 ns ± 0.0684 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.5475, -0.0708,  0.0126],
               [-0.9566,  2.0148,  0.2394]]),
    x: Batch(
           c: tensor([[ 0.7988, -1.0082, -1.7218, -0.3509],
                      [-1.3154, -0.4027, -2.8071,  0.8890],
                      [ 1.2666, -0.2290,  1.1835,  2.4907]]),
       ),
)

In [10]:
b.a

tensor([[ 0.5475, -0.0708,  0.0126],
        [-0.9566,  2.0148,  0.2394]])

In [11]:
%timeit b.a

58.3 ns ± 0.152 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1178, -0.5004, -2.1553],
               [-1.5320,  0.7189, -0.6567]]),
    x: Batch(
           c: tensor([[ 0.7988, -1.0082, -1.7218, -0.3509],
                      [-1.3154, -0.4027, -2.8071,  0.8890],
                      [ 1.2666, -0.2290,  1.1835,  2.4907]]),
       ),
)

In [13]:
%timeit b.a = new_value

491 ns ± 0.327 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

832 ns ± 0.609 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 22.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 259 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 560 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f5f05cd9700>
├── 'a' --> tensor([[[ 0.5475, -0.0708,  0.0126],
│                    [-0.9566,  2.0148,  0.2394]],
│           
│                   [[ 0.5475, -0.0708,  0.0126],
│                    [-0.9566,  2.0148,  0.2394]],
│           
│                   [[ 0.5475, -0.0708,  0.0126],
│                    [-0.9566,  2.0148,  0.2394]],
│           
│                   [[ 0.5475, -0.0708,  0.0126],
│                    [-0.9566,  2.0148,  0.2394]],
│           
│                   [[ 0.5475, -0.0708,  0.0126],
│                    [-0.9566,  2.0148,  0.2394]],
│           
│                   [[ 0.5475, -0.0708,  0.0126],
│                    [-0.9566,  2.0148,  0.2394]],
│           
│                   [[ 0.5475, -0.0708,  0.0126],
│                    [-0.9566,  2.0148,  0.2394]],
│           
│                   [[ 0.5475, -0.0708,  0.0126],
│                    [-0.9566,  2.0148,  0.2394]]])
└── 'x' --> <FastTreeValue 0x7f5f05cd9610>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 45.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f5fb4e39e50>
├── 'a' --> tensor([[ 0.5475, -0.0708,  0.0126],
│                   [-0.9566,  2.0148,  0.2394],
│                   [ 0.5475, -0.0708,  0.0126],
│                   [-0.9566,  2.0148,  0.2394],
│                   [ 0.5475, -0.0708,  0.0126],
│                   [-0.9566,  2.0148,  0.2394],
│                   [ 0.5475, -0.0708,  0.0126],
│                   [-0.9566,  2.0148,  0.2394],
│                   [ 0.5475, -0.0708,  0.0126],
│                   [-0.9566,  2.0148,  0.2394],
│                   [ 0.5475, -0.0708,  0.0126],
│                   [-0.9566,  2.0148,  0.2394],
│                   [ 0.5475, -0.0708,  0.0126],
│                   [-0.9566,  2.0148,  0.2394],
│                   [ 0.5475, -0.0708,  0.0126],
│                   [-0.9566,  2.0148,  0.2394]])
└── 'x' --> <FastTreeValue 0x7f5fb4da4670>
    └── 'c' --> tensor([[ 0.7988, -1.0082, -1.7218, -0.3509],
                        [-1.3154, -0.4027, -2.8071,  0.8890],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 396 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.4 µs ± 78.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.5475, -0.0708,  0.0126],
                [-0.9566,  2.0148,  0.2394]],
       
               [[ 0.5475, -0.0708,  0.0126],
                [-0.9566,  2.0148,  0.2394]],
       
               [[ 0.5475, -0.0708,  0.0126],
                [-0.9566,  2.0148,  0.2394]],
       
               [[ 0.5475, -0.0708,  0.0126],
                [-0.9566,  2.0148,  0.2394]],
       
               [[ 0.5475, -0.0708,  0.0126],
                [-0.9566,  2.0148,  0.2394]],
       
               [[ 0.5475, -0.0708,  0.0126],
                [-0.9566,  2.0148,  0.2394]],
       
               [[ 0.5475, -0.0708,  0.0126],
                [-0.9566,  2.0148,  0.2394]],
       
               [[ 0.5475, -0.0708,  0.0126],
                [-0.9566,  2.0148,  0.2394]]]),
    x: Batch(
           c: tensor([[[ 0.7988, -1.0082, -1.7218, -0.3509],
                       [-1.3154, -0.4027, -2.8071,  0.8890],
                       [ 1.2666, -0.2290,  1.1835,  2.4907]],
         

In [26]:
%timeit Batch.stack(batches)

78.1 µs ± 113 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.5475, -0.0708,  0.0126],
               [-0.9566,  2.0148,  0.2394],
               [ 0.5475, -0.0708,  0.0126],
               [-0.9566,  2.0148,  0.2394],
               [ 0.5475, -0.0708,  0.0126],
               [-0.9566,  2.0148,  0.2394],
               [ 0.5475, -0.0708,  0.0126],
               [-0.9566,  2.0148,  0.2394],
               [ 0.5475, -0.0708,  0.0126],
               [-0.9566,  2.0148,  0.2394],
               [ 0.5475, -0.0708,  0.0126],
               [-0.9566,  2.0148,  0.2394],
               [ 0.5475, -0.0708,  0.0126],
               [-0.9566,  2.0148,  0.2394],
               [ 0.5475, -0.0708,  0.0126],
               [-0.9566,  2.0148,  0.2394]]),
    x: Batch(
           c: tensor([[ 0.7988, -1.0082, -1.7218, -0.3509],
                      [-1.3154, -0.4027, -2.8071,  0.8890],
                      [ 1.2666, -0.2290,  1.1835,  2.4907],
                      [ 0.7988, -1.0082, -1.7218, -0.3509],
                      [-1.3154, -

In [28]:
%timeit Batch.cat(batches)

144 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

331 µs ± 1.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
